In [4]:
import pybaseball
from pybaseball import statcast_pitcher
from pybaseball import playerid_lookup



Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,CU,2019-05-07,74.5,-1.1053,6.0146,Mike Fiers,553993,571666,strikeout,swinging_strike_blocked,...,2,0,0,2,0,2,0,2,Standard,Standard
1,FF,2019-05-07,90.4,-0.9822,6.1465,Mike Fiers,553993,571666,NaN,ball,...,2,0,0,2,0,2,0,2,Standard,Standard
2,FF,2019-05-07,90.3,-0.9776,6.1298,Mike Fiers,553993,571666,NaN,ball,...,2,0,0,2,0,2,0,2,Standard,Standard
3,FF,2019-05-07,90.2,-0.9456,6.1670,Mike Fiers,553993,571666,NaN,foul,...,2,0,0,2,0,2,0,2,Standard,Standard
4,FT,2019-05-07,88.1,-1.3783,5.9430,Mike Fiers,553993,571666,NaN,called_strike,...,2,0,0,2,0,2,0,2,Standard,Standard
5,CH,2019-05-07,84.1,-1.6764,5.8684,Mike Fiers,458015,571666,field_out,hit_into_play,...,2,0,0,2,0,2,0,2,Infield shift,Standard
6,CH,2019-05-07,83.2,-1.5153,5.8974,Mike Fiers,458015,571666,NaN,called_strike,...,2,0,0,2,0,2,0,2,Infield shift,Standard
7,FF,2019-05-07,89.8,-1.0146,6.1690,Mike Fiers,458015,571666,NaN,ball,...,2,0,0,2,0,2,0,2,Infield shift,Standard
8,FF,2019-05-07,89.4,-1.1568,6.1746,Mike Fiers,458015,571666,NaN,ball,...,2,0,0,2,0,2,0,2,Infield shift,Standard
9,FF,2019-05-07,89.7,-1.0413,6.2373,Mike Fiers,458015,571666,NaN,called_strike,...,2,0,0,2,0,2,0,2,Infield shift,Standard


In [8]:
Fiers.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [15]:
from pybaseball import batting_stats


# retrieve data on only players who have 50+ plate appearances this year
data = batting_stats(2017, qual=50)

AttributeError: 'DataFrame' object has no attribute 'UZR'

In [20]:
data.columns

Index(['Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B',
       ...
       'wSL/C (pi)', 'wXX/C (pi)', 'O-Swing% (pi)', 'Z-Swing% (pi)',
       'Swing% (pi)', 'O-Contact% (pi)', 'Z-Contact% (pi)', 'Contact% (pi)',
       'Zone% (pi)', 'Pace (pi)'],
      dtype='object', length=287)